In [ ]:
%pip install comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.9/561.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.2
    Uninstalling websocket-client-1.6.2:
      Successfully uninstalled websocket-client-1.6.2
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1


In [17]:
import comet_ml
comet_ml.init(project_name="Prueba comet")

In [18]:
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint
import gzip
import pickle
import numpy as np

In [25]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/eduardo-macavity/prueba-comet/1a8e17f4861c4215a876e86103164bb0



In [19]:
modelo_cargado = tf.keras.models.load_model('best_model.hdf5')

In [20]:
#Cargamos los datos
f = gzip.open('mnist.pkl.gz', 'rb')
dataset = pickle.load(f,encoding="latin1")

In [21]:


dat=np.array(dataset)
print(dat[1,1].shape)
(x_train, y_train), (x_val, y_val), (x_test, y_test) = dataset



(10000,)


<ipython-input-21-57f718d27ce6>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dat=np.array(dataset)


In [22]:


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')



In [23]:

num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [32]:


parameters = {
    "batch_size": 128,
    "epochs": 30,
    "optimizer": "adam",
    "loss": "binary_crossentropy",
}

experiment.log_parameters(parameters)

In [33]:
model = Sequential(modelo_cargado)
model.add(Dense(512, activation='sigmoid', input_shape=(784,)))
model.add(Dense(600,activation='tanh'))
model.add(Dense(200,activation='selu'))
model.add(Dense(600,activation='relu'))
model.add(Dense(100,activation='elu'))
model.add(Dense(600,activation='sigmoid'))
model.add(Dense(200, activation='elu', kernel_regularizer=regularizers.L2(l2=1e-4)) ) #Regularizacion L2
model.add(Dense(500,activation='exponential'))
model.add(Dense(num_classes, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 10)                1376730   
                                                                 
 dense_9 (Dense)             (None, 512)               5632      
                                                                 
 dense_10 (Dense)            (None, 600)               307800    
                                                                 
 dense_11 (Dense)            (None, 200)               120200    
                                                                 
 dense_12 (Dense)            (None, 600)               120600    
                                                                 
 dense_13 (Dense)            (None, 100)               60100     
                                                                 
 dense_14 (Dense)            (None, 600)              

In [34]:
model.compile(loss=parameters['loss'],optimizer=parameters['optimizer'],metrics=['accuracy'])


In [35]:
model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    )

Epoch 1/30
391/391 [==============================] - 28s 65ms/step - loss: 0.3572 - accuracy: 0.1033 - val_loss: 0.3466 - val_accuracy: 0.0974
Epoch 2/30
391/391 [==============================] - 23s 59ms/step - loss: 0.3415 - accuracy: 0.1029 - val_loss: 0.3367 - val_accuracy: 0.1032
Epoch 3/30
391/391 [==============================] - 23s 60ms/step - loss: 0.3334 - accuracy: 0.1029 - val_loss: 0.3309 - val_accuracy: 0.0974
Epoch 4/30
391/391 [==============================] - 24s 61ms/step - loss: 0.3292 - accuracy: 0.1059 - val_loss: 0.3280 - val_accuracy: 0.1009
Epoch 5/30
391/391 [==============================] - 24s 62ms/step - loss: 0.3272 - accuracy: 0.1019 - val_loss: 0.3262 - val_accuracy: 0.1135
Epoch 6/30
391/391 [==============================] - 23s 59ms/step - loss: 0.3261 - accuracy: 0.1067 - val_loss: 0.3257 - val_accuracy: 0.0974
Epoch 7/30
391/391 [==============================] - 27s 68ms/step - loss: 0.3257 - accuracy: 0.1071 - val_loss: 0.3255 - val_accuracy:

In [36]:
score = model.evaluate(x_test, y_testc, verbose=1) #evaluar la eficiencia del modelo
print(score)
a=model.predict(x_test) #predicción de la red entrenada
print(a.shape)
print(a[1])
print("resultado correcto:")
print(y_testc[1])


313/313 [==============================] - 3s 11ms/step - loss: 0.3251 - accuracy: 0.1135
[0.32514017820358276, 0.11349999904632568]
313/313 [==============================] - 3s 8ms/step
(10000, 10)
[0.10839035 0.11797436 0.10219676 0.10944629 0.09515241 0.09895652
 0.1027865  0.10888895 0.10056282 0.10443575]
resultado correcto:
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [37]:
experiment.log_model("MNIST1", "best_model.hdf5")

{'web': 'https://www.comet.com/api/asset/download?assetId=6c2cc1629337454a9b7d13d60b1d82a8&experimentKey=1a8e17f4861c4215a876e86103164bb0',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=6c2cc1629337454a9b7d13d60b1d82a8&experimentKey=1a8e17f4861c4215a876e86103164bb0',
 'assetId': '6c2cc1629337454a9b7d13d60b1d82a8'}

In [38]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/eduardo-macavity/prueba-comet/1a8e17f4861c4215a876e86103164bb0
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [32]                 : (0.10189999639987946, 0.3930400013923645)
COMET INFO:     batch_accuracy [1282]         : (0.0703125, 0.59375)
COMET INFO:     batch_loss [1282]             : (0.19660446047782898, 1.168004035949707)
COMET INFO:     epoch_duration [32]           : (22.042435695999984, 29.855729190000034)
COMET INFO:     loss [32]                     : (0.2672901749610901, 0.3572080731391907)
COMET INFO:     val_accuracy [32]             : (0.0957999974489212, 0.5720000267028809)
COMET INFO:     

In [39]:
#Para guardar el modelo en disco
#model.save("red.h5")

In [40]:
#para cargar la red:
#modelo_cargado = tf.keras.models.load_model('red.h5')